# SQL-Python Integration

## Demonstrating Python vs SQL

The following example illustrates the power of SQL for manipulating tabular data. It uses a table of synthetic sales data with 5000 rows and 11 columns. Various operations are demonstrated in order of increasing complexity. The Python and SQL solutions provided for each allow for direct comparison.

Run the following cell first to get the required data files. Though you may not know the methods used, you should, at this point, be able to follow the code and get a sense of what it does.

In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

def download_if_missing(url, filename, data_dir = Path('data')):
    """
    Download a file from URL if not present in local data directory.
    
    Args:
        url: Source URL to download from
        filename: Name to save file as locally
        data_dir: Path to data directory (default: Path('data'))
    
    Returns:
        Path: Path object pointing to the local file
    """

    # make sure a data subdirectory exists
    data_dir.mkdir(exist_ok=True)

    # build a full Path objecct
    filepath = data_dir / filename

    if not filepath.exists():
        # there is no file in the data directory called filename
        print(f"Downloading {filename} to {data_dir}/...")
        try:
            urlretrieve(url, filepath)
            print("Download complete!")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filepath} exists, skipping download.")

    return filepath

# define constants
DATA_DIR = Path('data')
BASE_URL = 'https://raw.githubusercontent.com/olearydj/INSY3010-Fall24/main/notebooks/data/'
CSV_FILE = '5k-sales.csv'
DB_FILE = '5k-sales.db'

# Download CSV if needed
csv_path = download_if_missing(BASE_URL + CSV_FILE, CSV_FILE, DATA_DIR)

# Delete DB if exists
db_path = DATA_DIR / DB_FILE
db_path.unlink(missing_ok=True)


---
Import libraries, including Pandas (alternative method for building a table).

In [ ]:
# sqlite3 is included with python
import sqlite3
import os
import pandas as pd

---
Create connection and cursor, build the table using **sqlite**. Then populate it with data using **Pandas** `read_csv` and `to_sql` methods.

In [ ]:
# create the connector and cursor
con = sqlite3.connect(db_path)
cur = con.cursor()

# read the csv and create a SQLite table using the Pandas library
data = pd.read_csv(csv_path)
data.to_sql('sales', con, if_exists='replace', index=False)

# commit the changes
con.commit()

Check the outcome by executing a simple query and fetching all the results.

In [ ]:
# fetchall returns all lines of the result as a list of tuples
cur.execute("SELECT * FROM sales LIMIT 1").fetchall()

---
### Display rows of data...

Not very helpful here as most of the work was in getting valid input and printing it nicely.

In [ ]:
res = cur.execute("SELECT * FROM sales LIMIT 5")
# iterate through res to get rows of result as tuples
for row in res:
    for cell in row[:2]:
        print(cell, end='  ')
    print()

Double-check the column names. Stored in `res.description`.

In [ ]:
# re-using the previous results, inspecting description attribute
print('Columns in sales table:')
for col in res.description:
    print(col[0])  # just first item in single element tuple

---
### List unique values in a column...

Here are some Python functions to achieve the desired result. This assumes the data is represented as a list of lists, with the column names in the first row.

```python
def get_col_number(data, col_name):
    '''Return the index number of the named column in row 0'''
    col_num = 0
    for name in data[0]:  # first row has column names
        if name == col_name:
            return col_num
        col_num += 1
    return None  # not found

def get_col_data(data, col_num):
    '''returns selected column of sample data as list'''
    col_data = []
    for row in data[1:]:
        col_data.append(row[col_num])
    return col_data

def get_unique_vals(data):
    '''returns a sorted list of the unique values in data'''
    return sorted(list(set(data)))

# example usage
# get the unique values in the NAME column of YOUR_DATA
column_num = get_col_number(YOUR_DATA, NAME)
column_data = get_col_data(YOUR_DATA, column_num)
unique_vals = get_unique_vals(column_data)
```

Here we see the benefits of SQL, which can accomplish the same with a single line query:

In [ ]:
# demonstrate for unique values in Region
res = cur.execute("SELECT DISTINCT Region from sales ORDER BY Region")
for region in res:
    print(region[0])

---
### Calculate average units per order for selected item type...  

Here is one way to achieve this in Python:

```python
units = 0
num = 0
for order in data:
    if order[3] == item_name:
        units += order[8]
        num += 1
avg_units = units / num
print(f"\nThe average quantity for each {item_name} order is {avg_units:.0f} units.")
```

SQL can do the same with a relatively simple query.

In [ ]:
# average cost by unit type
SQL = "SELECT Item_Type, AVG(Unit_Cost) FROM sales GROUP BY Item_Type"
res = cur.execute(SQL)
for r in res:
    # fancy f-strings
    print(f"{r[0]:17.17}$ {r[1]:>6.2f}")

---
### Identify the most profitable order for each country in selected region...

Things are getting more complicated in Python now. Note this function uses one defined above to get the index for necessary columns by name.

```python
def find_most_profitable_orders(data, target_region):
    """Find the most profitable order for each country in a region
    
    Args:
        data: list of lists with column headers in row 0
        target_region: region to analyze
    Returns:
        dict: country -> [order_id, profit] mappings
    """
    # Get column positions from headers
    region_col = get_col_number(data, "Region")
    country_col = get_col_number(data, "Country")
    units_col = get_col_number(data, "Units_Sold")
    price_col = get_col_number(data, "Unit_Price")
    cost_col = get_col_number(data, "Unit_Cost")
    order_col = get_col_number(data, "Order_ID")
    
    # Dictionary to store results
    country_profits = {}
    
    # Check each row (skip headers)
    for row in data[1:]:
        # Only look at rows for our target region
        if row[region_col] == target_region:
            country = row[country_col]
            # Calculate profit for this order
            profit = row[units_col] * (row[price_col] - row[cost_col])
            order_id = row[order_col]
            
            # If first order for country or higher profit, store it
            if country not in country_profits:
                country_profits[country] = [order_id, profit]
            elif profit > country_profits[country][1]:
                country_profits[country] = [order_id, profit]
    
    return country_profits

# Example usage:
region = "Asia"
results = find_most_profitable_orders(data, region)
print("Most profitable orders by country:")
for country, (order_id, profit) in results.items():
    print(f"{country}: Order {order_id}, Profit ${profit:,.2f}")
```

In SQL, this requires a sub-query to select the desired columns of the output. Still the main query is much more straightforward.

In [ ]:
# use a nested query
SQL = (
    # 2 - select only the desired columns of that result
    "SELECT Order_ID, Country, Item_Type, Sales_Channel "
    "FROM("
        # 1 - get all the original columns plus max Profit for each country
        "SELECT *, MAX(Units_Sold * (Unit_Price - Unit_Cost)) AS Profit "
        "FROM sales "
        "WHERE Region = 'Asia' "
        "GROUP BY Country"
    ")"
)

cur.execute(SQL).fetchall()

---
Close the connection!

In [ ]:
con.close()

---

Auburn University / Industrial and Systems Engineering  
INSY 3010 / Programming and Databases for ISE / Fall 2024  
© Copyright 2024, Danny J. O'Leary.  
For licensing, attribution, and information: [GitHub INSY3010-Fall24](https://github.com/olearydj/INSY3010-Fall24)
